In [ ]:
import os
import copy
import pickle
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer

#torch packages
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#transformer packages
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel
from transformers import AdamW
from transformers import get_scheduler
from transformers import logging
logging.set_verbosity_error() #turn off bert warning
logging.set_verbosity_warning() #turn off bert warning

In [ ]:
def set_seed(seed):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
def generate_fold_num_for_dataset(data, num_fold):
    skf = StratifiedKFold(n_splits=num_fold, shuffle=True)
    for fold, ( _, val_) in enumerate(skf.split(X=data, y=data.worker)):
        data.loc[val_ , "kfold"] = int(fold)
    data["kfold"] = data["kfold"].astype(int)
    return data

class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer, max_length, use_tfidf=False, tfidf_matrix=None):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.more_toxic = df['more_toxic'].values
        self.less_toxic = df['less_toxic'].values
        self.use_tfidf = use_tfidf
        if use_tfidf:
            self.more_toxic_tfidf_idx = df['more_toxic_tfidf_idx'].values
            self.less_toxic_tfidf_idx = df['less_toxic_tfidf_idx'].values
            self.tfidf_matrix = tfidf_matrix

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        more_toxic = self.more_toxic[index]
        less_toxic = self.less_toxic[index]
        inputs_more_toxic = self.tokenizer.encode_plus(
                                more_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        inputs_less_toxic = self.tokenizer.encode_plus(
                                less_toxic,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        target = 1
        
        more_toxic_ids = inputs_more_toxic['input_ids']
        more_toxic_mask = inputs_more_toxic['attention_mask']        
        less_toxic_ids = inputs_less_toxic['input_ids']
        less_toxic_mask = inputs_less_toxic['attention_mask']
        
        if self.use_tfidf:
            more_toxic_tfidf_idx = self.more_toxic_tfidf_idx[index]
            less_toxic_tfidf_idx = self.less_toxic_tfidf_idx[index]
            more_toxic_tfidf = self.tfidf_matrix[more_toxic_tfidf_idx]
            less_toxic_tfidf = self.tfidf_matrix[less_toxic_tfidf_idx]
            return {
                'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
                'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
                'more_toxic_tfidf': torch.tensor(more_toxic_tfidf, dtype=torch.long),
                'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
                'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
                'less_toxic_tfidf': torch.tensor(less_toxic_tfidf, dtype=torch.long),
                'target': torch.tensor(target, dtype=torch.long)
            }
        else:
            return {
                'more_toxic_ids': torch.tensor(more_toxic_ids, dtype=torch.long),
                'more_toxic_mask': torch.tensor(more_toxic_mask, dtype=torch.long),
                'less_toxic_ids': torch.tensor(less_toxic_ids, dtype=torch.long),
                'less_toxic_mask': torch.tensor(less_toxic_mask, dtype=torch.long),
                'target': torch.tensor(target, dtype=torch.long)
            }

In [ ]:
class NN(nn.Module):
    def __init__(self, bert_drop_out, HID_DIM=768, tfidf_len=0, use_tfidf=False):
        super().__init__()
        if use_tfidf:
            self.net = nn.Sequential(
                nn.Dropout(p=bert_drop_out), #dropout for bert
                
                nn.Linear(768+tfidf_len, 1)
            )
        else:
            self.net = nn.Sequential(
                nn.Dropout(p=bert_drop_out), #dropout for bert
                nn.Linear(768, 1)
            )
            
    def forward(self, x):
        score = self.net(x)
        return score
                
class JigsawModel(nn.Module):
    def __init__(self, BERT, NN):
        super(JigsawModel, self).__init__()
        self.bert = BERT
        self.fc = NN
        
    def forward(self, ids, mask, tfidf_vec=None, use_tfidf=False):        
        out = self.bert(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        if use_tfidf:
            fc_in = torch.cat(
                (out["pooler_output"], tfidf_vec), dim=1
            )
        else:
            fc_in = out["pooler_output"]
        outputs = self.fc(fc_in)
        return outputs

In [ ]:
def train_step_combine(
    model, criterion, optimizer, 
    train_loader, progress_bar, device, epoch, use_tfidf = False
):
    y_preds = []
    epoch_loss = 0
    model.train()
    for i, data in enumerate(train_loader):
        more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.long)
        more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.long)
        less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.long)
        less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)

        optimizer.zero_grad()
        
        if use_tfidf:
            more_toxic_tfidf = data['more_toxic_tfidf'].to(device, dtype = torch.long)
            less_toxic_tfidf = data['less_toxic_tfidf'].to(device, dtype = torch.long)
            more_out = model(more_toxic_ids, more_toxic_mask, more_toxic_tfidf, use_tfidf=True)
            less_out = model(less_toxic_ids, less_toxic_mask, less_toxic_tfidf, use_tfidf=True)
        else:
            more_out = model(more_toxic_ids, more_toxic_mask)
            less_out = model(less_toxic_ids, less_toxic_mask)
            
        loss = criterion(more_out, less_out, targets)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
        for j in range(len(data['more_toxic_ids'])):
            y_preds.append([less_out[j].item(), more_out[j].item()])
        
        if progress_bar is not None:
            progress_bar.update(1)  
            
        print('[ Epoch {}: {}/{} ] loss:{:.3f}'.format(epoch, i+1, len(train_loader), loss.item()), end='\r')
    
    df_score = pd.DataFrame(y_preds,columns=['less','more'])
    train_accuracy = validate_accuracy(df_score)         
    
    return df_score, train_accuracy, epoch_loss / len(train_loader) # return loss


def validate_all_combine(
    model, criterion, 
    valid_loader, device, use_tfidf=False
):
    epoch_loss = 0
    y_preds = []
    
    model.eval()
    with torch.no_grad():
        for data in valid_loader:
            more_toxic_ids = data['more_toxic_ids'].to(device, dtype = torch.long)
            more_toxic_mask = data['more_toxic_mask'].to(device, dtype = torch.long)
            less_toxic_ids = data['less_toxic_ids'].to(device, dtype = torch.long)
            less_toxic_mask = data['less_toxic_mask'].to(device, dtype = torch.long)
            targets = data['target'].to(device, dtype=torch.long)
            
            if use_tfidf:
                more_toxic_tfidf = data['more_toxic_tfidf'].to(device, dtype = torch.long)
                less_toxic_tfidf = data['less_toxic_tfidf'].to(device, dtype = torch.long)
                more_out = model(more_toxic_ids, more_toxic_mask, more_toxic_tfidf, use_tfidf=True)
                less_out = model(less_toxic_ids, less_toxic_mask, less_toxic_tfidf, use_tfidf=True)
            else:
                more_out = model(more_toxic_ids, more_toxic_mask)
                less_out = model(less_toxic_ids, less_toxic_mask)
            
            loss = criterion(more_out, less_out, targets)

            epoch_loss += loss.item()
            for i in range(len(data['more_toxic_ids'])):
                y_preds.append([less_out[i].item(), more_out[i].item()])
        df_score = pd.DataFrame(y_preds,columns=['less','more'])
        accuracy = validate_accuracy(df_score)
    return df_score, accuracy, (epoch_loss / len(valid_loader))

def validate_accuracy(df_score):
    return len(df_score[df_score['less'] < df_score['more']]) / len(df_score)

def return_wrong_text(df_score, df_valid):
    df_score_text = pd.concat((df_valid.reset_index().drop('index',axis=1),df_score),axis=1)
    return df_score_text[df_score_text['less'] > df_score_text['more']]

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
def plot_loss_plot(train_losses, valid_losses):
    plt.plot(train_losses,label="Training")
    plt.plot(valid_losses,label="Validation")
    plt.title("Loss plot")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    
def plot_acc_plot(train_accs, valid_accs):
    plt.plot(train_accs,label="Training")
    plt.plot(valid_accs,label="Validation")
    plt.title("Accuracy plot")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

In [ ]:
def remove_duplicates(df, used_col):
    """Combine `less_toxic` text and `more_toxic` text,
    then remove duplicate pair of comments while keeping the last pair
    """
    df["combine"] = df["less_toxic"] + df["more_toxic"]
    df = df.drop_duplicates(subset=used_col, keep="last")
    return df

def create_corpus(df_train):
    all_corpus = df_train["more_toxic"].to_list()
    all_corpus += df_train["less_toxic"].to_list()
    #remove duplicates
    all_corpus = list(
        set(all_corpus)
    )
    return all_corpus

def create_mapping_dict(corpus):
    idx = np.arange(len(corpus))
    sentence2idx = dict(
        zip(corpus, idx)
    )
    return sentence2idx

def tokenize_by_bert_tokenizer(corpus, tokenizer):
    corpus_tokenized = [
        tokenizer.tokenize(sentence) for sentence in corpus
    ]
    return corpus_tokenized

def identity_tokenizer(text):
    return text

def corpus2tfidf(corpus_tokenized):
    tfidf = TfidfVectorizer(preprocessor=' '.join, tokenizer=identity_tokenizer)    
    tfidf_matrix_sparse = tfidf.fit_transform(corpus_tokenized)
    tfidf_matrix = tfidf_matrix_sparse.toarray()
    return tfidf, tfidf_matrix
    
def construct_tfidf_matrix(df_train, tokenizer):
    corpus = create_corpus(df_train)
    sentence2idx = create_mapping_dict(corpus)
    corpus_tokenized = tokenize_by_bert_tokenizer(corpus, tokenizer)
    tfidf_obj, tfidf_matrix = corpus2tfidf(corpus_tokenized)
    return sentence2idx, tfidf_obj, tfidf_matrix

In [ ]:
set_seed(5080)
data_train = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
data_test = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_train2 = pd.read_csv("../input/jigsaw-training-with-classification/classification_data.csv")

#load model and tokenizer
# PRETRAINED_MODEL_NAME = "GroNLP/hateBERT"
# bert_tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
PRETRAINED_MODEL_NAME = "roberta-base"
bert_tokenizer = RobertaTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [ ]:
data_train3 = data_train2.drop(columns = ['Unnamed: 0','less_score','more_score'])
data_train3['worker'] = [-1] * len(data_train3)
data_train = data_train.append(data_train3)

In [ ]:
data_train

# Remove Useless Words by Jack

In [ ]:
non_toxic = [' article ',' will ',' page ',' people ',' one ',' edit ',' wikipedia ',' talk ',' user ',' comment ',' know ',' really ',' think ',' Please ',' make ' ,' someone ',' thing ',' many' ,' name ',' said ',' time ',' need ',' look ',' editor ',' china ',' world ',' see ',' source ',' well ',' Fan ', ' fact ', ' Thank ',' wiki']

In [ ]:
len(non_toxic)

In [ ]:
# columns = ['less_toxic','more_toxic']

# for col in columns:
#     result = []
#     for sentense in data_train[col].to_list():
#         temp = sentense
#         for word in non_toxic:
#             temp = temp.replace(word,' ')
#         result.append(temp)
#     data_train[col] = result

In [ ]:
# columns = ['text']

# for col in columns:
#     result = []
#     for sentense in data_test[col].to_list():
#         temp = sentense
#         for word in non_toxic:
#             temp = temp.replace(word,' ')
#         result.append(temp)
#     data_test[col] = result

# TFIDF preprocessing

In [ ]:
#text preprocessing
data_train_removed = remove_duplicates(data_train, "combine") #(30108, 4) -> (15410, 4)

#construct tfidf一定要用removed!
sentence2idx, tfidf_obj, tfidf_matrix = construct_tfidf_matrix(data_train_removed, bert_tokenizer)
data_train["less_toxic_tfidf_idx"] = data_train["less_toxic"].apply(lambda x: sentence2idx[x])
data_train["more_toxic_tfidf_idx"] = data_train["more_toxic"].apply(lambda x: sentence2idx[x])

In [ ]:
with open('./tfidf_roberta_obj.pickle', 'wb') as f:
    pickle.dump(tfidf_obj, f)

In [ ]:
print(tfidf_obj)
print(tfidf_matrix.shape)

# Dataset and Dataloader

In [ ]:
#paramters settings
train_valid_ratio = 0.25
max_token_length = 128 #代表最多放入BERT的token長度
train_batch_size = 32
valid_batch_size = 64
use_tfidf = False

df_train, df_valid = train_test_split(
    data_train, test_size = train_valid_ratio
)


if use_tfidf:
    tfidf_len = tfidf_matrix.shape[1]
    train_dataset = JigsawDataset(
        df_train, tokenizer=bert_tokenizer, 
        max_length=max_token_length , use_tfidf=use_tfidf, tfidf_matrix=tfidf_matrix
     )
    valid_dataset = JigsawDataset(
        df_valid, tokenizer=bert_tokenizer, 
        max_length=max_token_length , use_tfidf=use_tfidf, tfidf_matrix=tfidf_matrix
    )
else:
    train_dataset = JigsawDataset(
        df_train, tokenizer=bert_tokenizer, 
        max_length=max_token_length #, use_tfidf=use_tfidf, tfidf_matrix=tfidf_matrix
     )
    valid_dataset = JigsawDataset(
        df_valid, tokenizer=bert_tokenizer, 
        max_length=max_token_length #, use_tfidf=use_tfidf, tfidf_matrix=tfidf_matrix
    )

train_loader = DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True,
    num_workers=2)
valid_loader = DataLoader(
    valid_dataset, batch_size=valid_batch_size, shuffle=False,
    num_workers=2)

# Modeling

#### best settings (until 1/12 10:30)
- PRETRAINED_MODEL_NAME = "roberta-base"
- train_valid_ratio = 0.25
- max_token_length = 128
- batch_size = 32
- LR = 1e-4
- WD = 1e-6
- bert dropout rate = 0.3
- EPOCH = 10
- NO-TFIDF

In [ ]:
LR = 1e-4
WD = 1e-6
bert_drop_out = 0.3
# margin_list = [0.5]

# LR_list = [1e-5, 1e-4]
# WD_list = [0, 1e-6]
# bert_dropout_list = [0.2, 0.3]
max_patience = 3

EPOCH = 10
HID_DIM = 768
MARGIN = 0.5
DATE = "0113"
model_name = "roberta"


# for LR in LR_list:
#     for WD in WD_list:
#         for bert_drop_out in bert_dropout_list:

print(f"LR = {LR}, WD = {WD}, bert drop out = {bert_drop_out}")

####################################################MODEL SETTINGS########################################
bert = RobertaModel.from_pretrained(PRETRAINED_MODEL_NAME).to(device)
if use_tfidf:
    dnn = NN(bert_drop_out, HID_DIM, 
             tfidf_len, use_tfidf=True
            ).to(device)

else:
    dnn = NN(
        bert_drop_out, HID_DIM
        ).to(device)

dnn.apply(init_weights)
model = JigsawModel(bert, dnn)
trainable_params = list(model.parameters())
num_trainable_params = sum(p.numel() for p in trainable_params)

criterion = nn.MarginRankingLoss(margin=MARGIN)
optimizer = AdamW(
    trainable_params
    ,lr=LR,weight_decay=WD)

print(f"Total trainable parameters {num_trainable_params}")

num_training_steps = EPOCH * len(train_loader)
###########################################################################################################

####################################################Records################################################
train_accs = []
valid_accs = []
train_losses = []
valid_losses = []
best_valid_loss = np.inf
best_valid_acc = 0
best_epoch = 0
best_model = None
no_update = 0 #number of non-updated epochs
MODEL_DIR = f"./{DATE}_{model_name}_LR_{LR}_WD_{WD}_BDR_{bert_drop_out}.pth"
###########################################################################################################

# start training
#progress_bar = None
progress_bar = tqdm(range(num_training_steps))
for epoch in range(1, EPOCH+1):
    _, train_acc, train_loss = train_step_combine(model, criterion, optimizer, train_loader, progress_bar, device, epoch, use_tfidf)
    df_score, valid_acc, valid_loss = validate_all_combine(model, criterion, valid_loader, device, use_tfidf)
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f"Epoch {epoch}, Loss(Train/Valid) = {round(train_loss, 4)}/{round(valid_loss, 4)}, Accuracy(Train/Valid) = {round(train_acc*100, 3)}%/{round(valid_acc*100, 3)}%")

    if valid_acc > best_valid_acc:
        no_update = 0
        print(f"Saving model...")
        best_epoch = epoch
        best_valid_acc = valid_acc
        best_valid_loss = valid_loss
        best_model = model
        torch.save(
            {"BERT": best_model.bert.state_dict(),"NN": best_model.fc.state_dict()}
            ,MODEL_DIR
        )
    #early stopping
    else:
        no_update += 1 

    if no_update == max_patience:
        break

print(f"Best epoch: {best_epoch}, valid loss: {round(best_valid_loss, 4)}, valid acc: {round(best_valid_acc*100, 4)}%")

#compute acc for all dataset
all_dataset = JigsawDataset(
    data_train, tokenizer=bert_tokenizer, 
    max_length=max_token_length, use_tfidf=use_tfidf, tfidf_matrix=tfidf_matrix
)

all_loader = DataLoader(
    all_dataset, batch_size=valid_batch_size, 
    shuffle=True, num_workers=2
)

df_score, accuracy, valid_loss = validate_all_combine(model, criterion, all_loader, device, use_tfidf)
df_score_text = return_wrong_text(df_score, df_valid)
csv_out_path = f"./wrong_text_{DATE}_LR_{LR}_WD_{WD}_BDR_{bert_drop_out}.csv"
df_score_text.to_csv(csv_out_path)
print(f"Accuracy in all dataset set: {round(accuracy*100, 3)}%")

#Evaluation
plot_loss_plot(train_losses, valid_losses)
plot_acc_plot(train_accs, valid_accs)

In [ ]:
# all_dataset = JigsawDataset(
#     data_train, tokenizer=bert_tokenizer, 
#     max_length=max_token_length)

# all_loader = DataLoader(
#     train_dataset, batch_size=batch_size, shuffle=True,
#     num_workers=2)

# df_score, accuracy, valid_loss = validate_all(bert, dnn, criterion, all_loader, device)
# print(f"Accuracy in all dataset set: {round(accuracy*100, 3)}%")

# df_score_text = return_wrong_text(df_score, df_valid)
# df_score_text.to_csv("./wrong_text_1229_2.csv")
# print(df_score_text)

# plot_loss_plot(train_losses, valid_losses)
# plot_acc_plot(train_accs, valid_accs)